In [2]:
import pandas as pd
import numpy as np

import pymysql

from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:


host = "10.0.2.153"
port = 3306
user = "stephen01"
password = "stephen01@2023"
dbname = "remita_staging"

In [4]:
def get_query_results():
    with pymysql.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM SalaryHistory" 
        df = pd.read_sql_query(sql, conn)
        
    return df

In [5]:
salaries = get_query_results()

salaries.head(2)

/tmp/ipykernel_1728/703973372.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,Id,PhoneNo,BvnNo,CreatedDate,PaymentDate,Amount,AccountNumber,BankCode,CompanyName
0,1,2348130230131,2227134531,2022-06-10 11:01:23.422617,2022-02-25 13:33:46,155000.0,5012284010,023,None
1,2,2348130230131,2227134531,2022-06-10 11:01:23.422635,2022-01-25 13:33:46,155000.0,5012284010,023,None


In [6]:
salaries.loc[ : , 'payment_dates']  = [x.strftime('%Y-%m-%d') for x in salaries['PaymentDate']]

salaries['payment_dates'] = pd.to_datetime(salaries['payment_dates'])

salaries.loc[ : , 'year_month_salary_date'] = [x.strftime('%Y-%m') for x in salaries['PaymentDate']]

salaries.head(2)

,Id,PhoneNo,BvnNo,CreatedDate,PaymentDate,Amount,AccountNumber,BankCode,CompanyName,payment_dates,year_month_salary_date
0,1,2348130230131,2227134531,2022-06-10 11:01:23.422617,2022-02-25 13:33:46,155000.0,5012284010,023,None,2022-02-25,2022-02
1,2,2348130230131,2227134531,2022-06-10 11:01:23.422635,2022-01-25 13:33:46,155000.0,5012284010,023,None,2022-01-25,2022-01


In [7]:
# offset salarry payment dates to end of the month
from pandas.tseries.offsets import MonthEnd

salaries['offset_payment_dates'] = pd.to_datetime(salaries['payment_dates'], format="%Y%m") + MonthEnd(0)

salaries.head()

,Id,PhoneNo,BvnNo,CreatedDate,PaymentDate,Amount,AccountNumber,BankCode,CompanyName,payment_dates,year_month_salary_date,offset_payment_dates
0,1,2348130230131,2227134531,2022-06-10 11:01:23.422617,2022-02-25 13:33:46,155000.0,5012284010,023,None,2022-02-25,2022-02,2022-02-28
1,2,2348130230131,2227134531,2022-06-10 11:01:23.422635,2022-01-25 13:33:46,155000.0,5012284010,023,None,2022-01-25,2022-01,2022-01-31
2,3,2348130230131,2227134531,2022-06-10 11:01:23.422637,2021-12-25 13:33:46,155000.0,5012284010,023,None,2021-12-25,2021-12,2021-12-31
3,4,23440971951,2220087024,2022-06-10 12:21:09.295481,2022-02-25 00:00:00,1074455.0,0783839390,044,None,2022-02-25,2022-02,2022-02-28
4,5,23440971951,2220087024,2022-06-10 12:21:09.295481,2022-01-25 00:00:00,1074455.0,0783839390,044,None,2022-01-25,2022-01,2022-01-31


In [8]:
# remove duplicates
salaries.drop_duplicates(subset = ['BvnNo', 'PaymentDate', 'Amount','AccountNumber', 'BankCode'], inplace = True)


In [22]:
def government_employees_salaries_check_30_day_product(salaries, max_month):
    max_date = salaries['PaymentDate'].max()
    max_month = pd.to_datetime(max_date.strftime('%Y-%m'))
    last_3_months = salaries[salaries['offset_payment_dates'] >= max_month + relativedelta(
        months=-2)]
    monthly_salary_summaries = last_3_months.groupby(['BvnNo', 'year_month_salary_date'])['Amount'].sum().reset_index()
    monthly_salary_summaries = monthly_salary_summaries.groupby('BvnNo')['Amount'].mean().rename(
        'average_monthly_salary_last_3_months').reset_index()
    last_3_months_salaries_summaries = last_3_months.groupby(['BvnNo']).agg(
        count_of_payments_last_3_months=pd.NamedAgg('year_month_salary_date', 'nunique'),
        sum_of_salary_payments_last_3_months=pd.NamedAgg('Amount', 'sum'),
        minimum_salary_payment_last_3_months=pd.NamedAgg('Amount', 'min'),
        # median_salary_payment_last_6_months = pd.NamedAgg('Amount', 'median'),
        # mode_salary_payment_last_6_months = pd.NamedAgg('Amount', pd.Series.mode),
        earliest_salary_payment_date_last_3_months=pd.NamedAgg('payment_dates', 'min'),
        latest_salary_payment_date_last_3_months=pd.NamedAgg('payment_dates', 'max')
        ).reset_index()
    latest_payment_dates = last_3_months_salaries_summaries[['BvnNo', 'latest_salary_payment_date_last_3_months']]
    latest_payment_dates.loc[:, 'year_month_latest_salary_date'] = [x.strftime('%Y-%m') for x in latest_payment_dates[
        'latest_salary_payment_date_last_3_months']]
    last_3_months = last_3_months.merge(latest_payment_dates, on = 'BvnNo')
    latest_salary_payment_amount_last_3_months = last_3_months[last_3_months['year_month_latest_salary_date'] == last_3_months['year_month_salary_date']].groupby(
        'BvnNo')['Amount'].sum().rename('latest_salary_payment_amount_last_3_months').reset_index()
    last_3_months_salaries_summaries = pd.merge(last_3_months_salaries_summaries, monthly_salary_summaries, on='BvnNo')
    last_3_months_salaries_summaries = pd.merge(last_3_months_salaries_summaries,
                                                latest_salary_payment_amount_last_3_months, on='BvnNo')
    last_3_months_salaries_summaries['is_qualified_on_salaries'] = last_3_months_salaries_summaries.apply(
        lambda x: calculate_last_3_months_salaries_government(x), axis=1)
    last_3_months_salaries_summaries.rename(columns={'is_qualified_on_salaries': 'pay_day_is_qualified_on_salaries'}, inplace=True)

    return last_3_months_salaries_summaries




    

In [23]:
max_date = salaries['PaymentDate'].max()

#max_date = pd.to_datetime('today')

print(max_date)

2023-09-25 10:44:21


In [24]:
max_month = pd.to_datetime(max_date.strftime('%Y-%m'))

max_month

Timestamp('2023-09-01 00:00:00')

In [25]:
def calculate_last_3_months_salaries_government(df):

    count = df['count_of_payments_last_3_months']

    if count >= 2 :
        return True
    else:
        return False

In [26]:
t = government_employees_salaries_check_30_day_product(salaries, max_month)
t

/tmp/ipykernel_1728/580390205.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_payment_dates.loc[:, 'year_month_latest_salary_date'] = [x.strftime('%Y-%m') for x in latest_payment_dates[


,BvnNo,count_of_payments_last_3_months,sum_of_salary_payments_last_3_months,minimum_salary_payment_last_3_months,earliest_salary_payment_date_last_3_months,latest_salary_payment_date_last_3_months,average_monthly_salary_last_3_months,latest_salary_payment_amount_last_3_months,pay_day_is_qualified_on_salaries
0,22156181943,1,101786.07,101786.07,2023-07-27,2023-07-27,101786.07,101786.07,False
1,22231561608,3,476629.65,47233.45,2023-07-24,2023-09-25,158876.55,47233.45,True
2,22237205984,1,142902.99,4191.17,2023-07-27,2023-07-27,142902.99,142902.99,False
3,22279791960,1,96882.20,48441.10,2023-07-07,2023-07-28,96882.20,96882.20,False
4,22329236757,1,27443.75,27443.75,2023-07-31,2023-07-31,27443.75,27443.75,False
5,22329724760,1,47462.25,3892.25,2023-07-27,2023-07-27,47462.25,47462.25,False
6,22330767497,1,45871.15,45871.15,2023-07-24,2023-07-24,45871.15,45871.15,False
7,22354326386,1,699435.84,699435.84,2023-07-27,2023-07-27,699435.84,699435.84,False
8,22374136653,1,66000.00,33000.00,2023-07-03,2023-07-27,66000.00,66000.00,False
9,22383444220,1,33000.00,33000.00,2023-07-03,2023-07-03,33000.00,33000.00,False


In [27]:
def calculate_last_3_months_salaries_government(df):

    count = df['count_of_payments_last_3_months']

    if count >= 2 :
        return True
    else:
        return False

In [28]:
def calculate_additional_summaries(salaries_summaries: pd.DataFrame, loans: pd.DataFrame, time_period: int) -> pd.DataFrame:
    
    monthly_expected_repayments = loans.groupby('BvnNo')['RepaymentAmount'].sum().rename(
        'total_debt_value').reset_index()
    summaries = pd.merge(salaries_summaries, monthly_expected_repayments, on='BvnNo', how='left')

    summaries['total_debt_value'].fillna(0, inplace=True)

    summaries['DTI'] = round(
        summaries['total_debt_value'] / summaries[f'average_monthly_salary_last_{time_period}_months'], 1)
    summaries['DTI'].fillna(1, inplace=True) ## update to 0
    
    
    
    
    active_loans = loans[loans['OutstandingAmount'] > 0]
    all_running_loans = active_loans.groupby('BvnNo')['disbursement_dates'].nunique().rename(
        'count_of_all_running_loans').reset_index()

    summaries = pd.merge(summaries, all_running_loans, on='BvnNo', how='outer')
    summaries['count_of_all_running_loans'].fillna(0, inplace=True)

    
    
    summaries['latest_sal_ge_min_payment'] = summaries[f'latest_salary_payment_amount_last_{time_period}_months'] >= \
                                             summaries[
                                                 f'minimum_salary_payment_last_{time_period}_months']

In [ ]:
## 30 day product  - government employee
 
final_df.loc[(final_df['total_no_of_running_facilities'] <= 5) &
                 (final_df['maximum_days_in_arrears'] < 90) &
                 (final_df['total_overdue_amount'] < scoring_summaries['average_salary_payment_last_3_months']) &
                 (scoring_summaries['is_gvnt_employee'] == True) &
                 (final_df['no_of_institution_types'] <= 2), 'is_crb_qualified'] = True